# CCA

canonical correlation analysis

Let $X^{(1)}$ be a p by 1 vector and $X^{(2)}$ be a q by 1 vector, $p<=q$

Let $E(X^{1}) = \mu^1_{(p*1)}; Cov(X^1) = \Sigma_{11(p*p)}$
$E(X^{2}) = \mu^2_{(q*1)}; Cov(X^2) = \Sigma_{22(q*q)}$

and

$Cov(X^1,X^2) = \Sigma_{12} = \Sigma_{21}^T$,
$E[[X^1 - \mu^1][X^2 - \mu^2]^T]$ is a p by q matrix.

So:

<img src="ex.png" alt="Drawing" style="width: 300px;"/>
<img src="cov.png" alt="Drawing" style="width: 300px;"/>

Set linear combinations:
$U=a^T X^1, V = b^TX^2$

We can obtain:
\begin{align*}
Var(U) &= a^T Cov(X^1)a = a^T \Sigma_{11}a\\
Var(V) &= b^T Cov(X^2)b = b^T \Sigma_{22}b\\
Cov(U,V) &= a^T Cov(X^1,X^2)b = a^T \Sigma_{12}b
\end{align*}

We seek coefficient vectors a and b such that

$Corr(U,V) = \frac{a^T\Sigma_{12}b}{\sqrt{a^T\Sigma_{11}a}\sqrt{b^T\Sigma_{22}b}}$
is as large as possible.

Variance of U and V = 1.


Define:

kth canonical variate pair = the pair of linear combination $U_k$ and $V_k$ having
unit variances, which maximize the correlation among all ** choices uncorrelated with
the previous k - 1 canonical variate pairs.** Note: uncorrelated does not equal to independent!

<img src="eig.png" alt="Drawing" style="width: 300px;"/>

Result:

$Max_{a,b} Corr(U,V) = \rho^*_1$

attained by the linear combinations

$U_1 = e_1^T \Sigma_{11}^{-1/2}X^1$ and $V_1 = f_1^T \Sigma_{22}^{-1/2} X^2$

The canonical variates have the properties
$Var(U_k) = Var(V_k) = 1$

$Cov(U_k,U_l) = Cov(V_k,V_l) = Cov(U_k,V_l) = 0$ when $k != l$

### Canonical variates from standardized variables

$Z^1 = [Z_1^1,Z_2^1,...,Z_p^1]^T$ and

$Z^2 = [Z_1^2, Z_2^2,...,Z_q^2]^T$

Here, $Cov(Z^1) = \rho_{11}, Cov(Z^2) = \rho_{22}$, and

$Cov(Z^1,Z^2) = \rho_{12}=\rho_{21}^T$.

Then use
$\rho_{11}^{-1/2}\rho_{12}\rho_{22}^{-1}\rho_{21}\rho_{11}^{-1/2}$
find eigenvalues and eigenvectors. Eigenvalues are the same as before but the eigenvectors are different.

<img src="z.png" alt="Drawing" style="width: 300px;"/>

$a^T_k(X^1-\mu^1) = a_k^T V_{11}^{1/2}Z^1$
where $V_{11}$ is the diagonal matrix with ith element $Var(X_i^1)$.

### Interpretation:

Look at the linear combinatin: $U_1 = a_{11}X_1^1 + a_{12}X_2^1+..$
1. $Corr(U_1, X_1^1)$... Here the correlation != the coefficient

Note p.18:
$\rho_{U,X^1} = Corr(U,X^1) = I^{-1/2}Cov(U,X^1)V_{11}^{-1/2} = A\Sigma_{11}V_{11}^{-1/2}$

In [2]:
import numpy as np
from numpy import dot
import scipy.linalg as la
from sklearn.cross_decomposition import CCA

In [ ]:
def var_single(X):
    """
    Find variance matrix
    """
    v = np.cov(X)
    r = np.matrix('v,1;1,v')
    return r
    

http://users.stat.umn.edu/~helwig/notes/cancor-Notes.pdf
reference of R code

In [203]:
def cca(X,Y):
    """
    Canonical Correlatio Analysis
    
    Input:
    X: observation matrix X, every column is one data point
    Y: observation matrix Y, every column is one data point
    
    Output:
    basis in X space, basis in Y space, correlation
    """
    # find variance and covariance matrix
    if len(X) == 1:
        cov_xx = var_single(X)
    else:
        cov_xx = np.cov(X)
    if len(Y) == 1:
        cov_yy = var_single(Y)
    else:
        cov_yy = np.cov(Y)
    n = len(X)

    cov_xy = np.cov(X, Y)[:n,n:]    
    cov_yx = np.transpose(cov_xy)
    # eigen
    cov_xx_evalue,cov_xx_evector = la.eig(cov_xx)
    cov_xx_isqrt = dot(dot(cov_xx_evector,np.diag(1/np.sqrt(cov_xx_evalue))),np.transpose(cov_xx_evector))
    
    cov_yy_evalue, cov_yy_evector = la.eig(cov_yy)
    cov_yy_isqrt = dot(dot(cov_yy_evector,np.diag(1/np.sqrt(cov_yy_evalue))), np.transpose(cov_yy_evector))
    a = la.inv(cov_yy)
    # Xmat and Ymat
    Xmat = dot(dot(dot(dot(cov_xx_isqrt,cov_xy),la.inv(cov_yy)),cov_yx),cov_xx_isqrt)
    ymat = dot(dot(dot(dot(cov_yy_isqrt,cov_yx),la.inv(cov_xx)),cov_xy),cov_yy_isqrt)
    
    r1=la.eig(Xmat)
    r2=la.eig(Ymat)
    
    return r1,r2
    

In [15]:
a=[-11.25,7.43, 15.48, 2.27, -48.90, -15.13, 49.28, 4.7, 61.32, -268.95, 8488]
b=[-10.87, 7.45, 14.97, 1.97, -47.71, -14.46, 44.36, 5.1, 61.76, -273.02, 8399]
c=[-11.18, 7.44, 14.20, 1.97, -48.29, -14.81, 43.66, 5.2, 64.16, -263.20, 8328]
d=[-10.62, 7.38, 15.02, 2.03, -49.06, -14.72, 44.80, 4.9, 64.04, -285.11, 8306]
e=[-11.02, 7.43, 12.92, 1.97, -47.44, -14.40, 41.20, 5.2, 57.46, -256.64, 8286]
f=[-10.83, 7.72, 13.58, 2.12, -48.34, -14.18, 43.06, 4.9, 52.18, -274.07, 8272]
g=[-11.18, 7.05, 14.12, 2.06, -49.34, -14.39, 41.68, 5.7, 61.60, -291.20, 8216]
h=[-11.05, 6.95, 15.34, 2.00, -48.21, -14.36, 41.32, 4.8, 63.00, -265.86, 8189]
i=[-11.15, 7.12, 14.52, 2.03, -49.15, -14.66, 42.36,4.9, 66.46, -269.62, 8180]

In [16]:
data = np.vstack((a,b,c,d,e,f,g,h,i))
run100=data[:,0]
long_jump = data[:,1]
shot = data[:,2]
high_jump = data[:,3]
run400 = data[:,4]
hurdle = data[:,5]
discus = data[:,6]
pole_vault = data[:,7]
javelin = data[:,8]
run1500 = data[:,9]
score = data[:,10]

X: shot, discus, javelin, pole_vault

Y: run100,run400,run1500,hurdle,long_jump,high_jump

In [17]:
X = np.vstack((shot, discus, javelin,pole_vault))
Y = np.vstack((run100, run400,run1500,hurdle,long_jump,high_jump))

In [205]:
r1,r2=cca(X,Y)

In [206]:
r1[0]

array([ 0.60595938+0.j,  0.94686012+0.j,  1.00000000+0.j,  1.00000000+0.j])

In [207]:
np.sqrt(r1[0])

array([ 0.77843392+0.j,  0.97306738+0.j,  1.00000000+0.j,  1.00000000+0.j])

In [208]:
np.sqrt(r2[0])

array([  7.78433925e-01 +0.00000000e+00j,
         9.73067377e-01 +0.00000000e+00j,
         0.00000000e+00 +7.82039080e-09j,
         1.42187762e-08 +0.00000000e+00j,
         1.00000000e+00 +0.00000000e+00j,   1.00000000e+00 +0.00000000e+00j])

In [91]:
def CCA_Mardia(H1, H2, dim):
    # H1 and H2 are NxD matrices containing samples rowwise.
    # dim is the desired dimensionality of CCA space.
    
    d1 = H1.shape[0]
    d2 = H2.shape[0]
    N = H1.shape[1]
    
    # Remove mean
    m1 = np.mean(H1, axis=1)
    m2 = np.mean(H2, axis=1)
    H1 = H1 - np.reshape(m1,(d1,1))
    H2 = H2 - np.reshape(m2,(d2,1))
    
    S11 = (dot(H1,np.transpose(H1)))/(N-1)
    S22 = (dot(H2,np.transpose(H2)))/(N-1)
    S12 = (dot(H2,np.transpose(H1)))/(N-1)

    D1,V1 = la.eig(S11)
    D2,V2 = la.eig(S22)

    K11 = dot(dot(V1,np.diag(1/np.sqrt(D1))),np.transpose(V1))
    K22 = dot(dot(V2,np.diag(1/np.sqrt(D2))),np.transpose(V2))

    T = dot(dot(K22,S12),K11)
    U,D,V = np.linalg.svd(T)
    D = np.diag(D)
    A = dot(K11,np.transpose(V[0:dim,:]))
    B = dot(K22,np.transpose(U[0:dim,:]))
    D = D[0:dim]
    return A,B,D
    

In [97]:
CCA_Mardia(X,Y,4)

(array([[ 0.50316581+0.j, -0.06281345+0.j,  1.15987226+0.j, -1.65332289+0.j],
        [ 0.08255531+0.j,  0.43747610+0.j, -0.16197003+0.j,  0.21997911+0.j],
        [ 0.09600330+0.j, -0.07034461+0.j, -0.28120598+0.j,  0.12058579+0.j],
        [ 3.14622143+0.j,  0.95045772+0.j,  2.57226974+0.j,  0.45561675+0.j]]),
 array([[ -4.55381457+0.j,  -0.09639920+0.j,  -2.47922391+0.j,
          -2.28782268+0.j],
        [ -0.57626866+0.j,  -0.31004618+0.j,   0.25704203+0.j,
          -1.48204993+0.j],
        [ -0.06972073+0.j,   0.04958413+0.j,  -0.05069234+0.j,
           0.10586283+0.j],
        [ -1.48079963+0.j,  -0.90421183+0.j,   0.90758005+0.j,
           1.80893553+0.j],
        [  4.35535280+0.j,  -0.48420844+0.j,  -2.63676550+0.j,
           1.37758996+0.j],
        [-10.17707404+0.j,  -9.41804996+0.j,   0.30837624+0.j,
           2.20140784+0.j]]),
 array([[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        ,  0.        ],
        [ 0.  

As we can see, both method yields same canonical correlation:

**1,1,0.973,0.778**

# Kernel Canonical Correlation Analysis

Kernel CCA offers an alternative solution by first projecting the data into a higher-dimensional feature space.

$\Phi: x = (x_1,...,x_m) \to \Phi(x) = (\Phi_1(x),...,\Phi_N(x)) (m<N)$

Kernel is known as the kernel trick. A kernel is a function K, such that for all x,z in X, 

$K(x,z) = <\Phi(x)*\Phi(z)>$,

where $\Phi$ is a mapping from X to a feature space F.

We can rewrite the covariance matrix C using the data matrices X and Y, which have the sample vector as rows and are therefore of size m by N; we obtain

$C_{xx} = X'X, C_{xy}=X'Y$.

Then $w_{x(N*1)} = X_{N*m}'\alpha_{m*1}, w_{y(N*1)}=Y_{N*m}'\beta_{m*1}$.

So $\rho = max_{\alpha,\beta} \frac{\alpha'XX'YY'\beta}{\sqrt{\alpha'XX'XX'\alpha * \beta'YY'YY'\beta}}$

Let $K_x = XX', K_y = YY'$ be the kernel matrices, we have

$\rho = max_{\alpha,\beta} \frac{\alpha'K_xK_y\beta}{\sqrt{\alpha'K_x^2\alpha \beta'K_y^2\beta}}$

So it is equivalent to maxmizing the numerator subject to denominator = 1.

## regularization

It combine the PLS term with the KCCA term in the denominator, obtaining:

$\rho = max_{\alpha,\beta} \frac{\alpha'K_xK_y\beta}{\sqrt{(\alpha'K_x^2\alpha+k||w_x||^2)(\beta'K^2_y\beta+k||w_y||^2)}}
=max_{\alpha,\beta} \frac{\alpha'K_xK_y\beta}{\sqrt{(\alpha'K_x^2\alpha+k\alpha'K_x\alpha)(\beta'K^2_y\beta+k\beta'K_y\beta)}}$

obtain a standard eigenproblem of the form $Ax=\lambda x$.

## Partial Gram-Schmidt Orthogonalization

Use PGSO as matrix decomposition approach.

problem:
KCCA involves and N by N eigenvalue problem and so is challenging in both memory and time (O(N3)). Several approximation methods have been proposed.

# Deep Canonical Correlation Analysis

Deep CCA computes representations of the two views by passing them
through multiple stacked layers of nonlinear transformation.

<img src="dnet.png" alt="Drawing" style="width: 300px;"/>

The goal is to jointly learn parameters for both views $W_l^v. b_l^v$ such that $corr(f_1(X_1),f_2(X_2))$ is as high as possible.

$(\theta_1^*,\theta_2^*) = argmax_{\theta_1,\theta_2} corr(f_1(X_1),f_2(X_2))$

To find $(\theta_1^*,\theta_2^*)$, we follow the gradient of the correlation objective as estimated on the training data.

k = o, so the output is the number of top k components of H1 and H2.


### Use L-BFGS second-order optimization method.

### Non-saturating nonlinearity

Best results are obtained by using $g(y)=g^3/3+y, s(x)=g^{-1}(x)$

1. s is not bounded, and its derivative falls off much more gradually with x.
2. it derivative is a simple function of its value

Python code:https://github.com/VahidooX/DeepCCA

Let's look at the loss function: y_pred is ignored, that is the
reason it return **-corr. We want to maximize the correlation = minimize the loss(-corr).**

In [ ]:
import theano.tensor as T


def cca_loss(outdim_size, use_all_singular_values):
    """
    The main loss function (inner_cca_objective) is wrapped in this function due to
    the constraints imposed by Keras on objective functions
    """
    def inner_cca_objective(y_true, y_pred):
        """
        It is the loss function of CCA as introduced in the original paper. There can be other formulations.
        It is implemented by Theano tensor operations, and does not work on Tensorflow backend
        y_true is just ignored
        """

        r1 = 1e-4
        r2 = 1e-4
        eps = 1e-12
        o1 = o2 = y_pred.shape[1]//2

        # unpack (separate) the output of networks for view 1 and view 2
        H1 = y_pred[:, 0:o1].T
        H2 = y_pred[:, o1:o1+o2].T

        m = H1.shape[1]

        H1bar = H1 - (1.0 / m) * T.dot(H1, T.ones([m, m]))
        H2bar = H2 - (1.0 / m) * T.dot(H2, T.ones([m, m]))

        SigmaHat12 = (1.0 / (m - 1)) * T.dot(H1bar, H2bar.T)
        SigmaHat11 = (1.0 / (m - 1)) * T.dot(H1bar, H1bar.T) + r1 * T.eye(o1)
        SigmaHat22 = (1.0 / (m - 1)) * T.dot(H2bar, H2bar.T) + r2 * T.eye(o2)

        # Calculating the root inverse of covariance matrices by using eigen decomposition
        [D1, V1] = T.nlinalg.eigh(SigmaHat11)
        [D2, V2] = T.nlinalg.eigh(SigmaHat22)

        # Added to increase stability
        posInd1 = T.gt(D1, eps).nonzero()[0]
        D1 = D1[posInd1]
        V1 = V1[:, posInd1]
        posInd2 = T.gt(D2, eps).nonzero()[0]
        D2 = D2[posInd2]
        V2 = V2[:, posInd2]

        SigmaHat11RootInv = T.dot(T.dot(V1, T.nlinalg.diag(D1 ** -0.5)), V1.T)
        SigmaHat22RootInv = T.dot(T.dot(V2, T.nlinalg.diag(D2 ** -0.5)), V2.T)

        Tval = T.dot(T.dot(SigmaHat11RootInv, SigmaHat12), SigmaHat22RootInv)

        if use_all_singular_values:
            # all singular values are used to calculate the correlation
            corr = T.sqrt(T.nlinalg.trace(T.dot(Tval.T, Tval)))
        else:
            # just the top outdim_size singular values are used
            [U, V] = T.nlinalg.eigh(T.dot(Tval.T, Tval))
            U = U[T.gt(U, eps).nonzero()[0]]
            U = U.sort()
            corr = T.sum(T.sqrt(U[0:outdim_size]))

        return -corr

    return inner_cca_objective


## Model

it uses sigmoid activation function instead of the one in the paper.

In [ ]:
from keras.layers import Dense, Merge
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.regularizers import l2
from objectives import cca_loss


def create_model(layer_sizes1, layer_sizes2, input_size1, input_size2,
                    learning_rate, reg_par, outdim_size, use_all_singular_values):
    """
    builds the whole model
    the structure of each sub-network is defined in build_mlp_net,
    and it can easily get substituted with a more efficient and powerful network like CNN
    """
    view1_model = build_mlp_net(layer_sizes1, input_size1, reg_par)
    view2_model = build_mlp_net(layer_sizes2, input_size2, reg_par)

    model = Sequential()
    model.add(Merge([view1_model, view2_model], mode='concat'))

    model_optimizer = RMSprop(lr=learning_rate)
    model.compile(loss=cca_loss(outdim_size, use_all_singular_values), optimizer=model_optimizer)

    return model


def build_mlp_net(layer_sizes, input_size, reg_par):
    model = Sequential()
    for l_id, ls in enumerate(layer_sizes):
        if l_id == 0:
            input_dim = input_size
        else:
            input_dim = []
        if l_id == len(layer_sizes)-1:
            activation = 'linear'
        else:
            activation = 'sigmoid'

        model.add(Dense(ls, input_dim=input_dim,
                                activation=activation,
                                kernel_regularizer=l2(reg_par)))
    return model

In [ ]:
if __name__ == '__main__':
    ############
    # Parameters Section

    # the path to save the final learned features
    save_to = './new_features.gz'

    # the size of the new space learned by the model (number of the new features)
    outdim_size = 10

    # size of the input for view 1 and view 2
    input_shape1 = 784
    input_shape2 = 784

    # number of layers with nodes in each one
    layer_sizes1 = [1024, 1024, 1024, outdim_size]
    layer_sizes2 = [1024, 1024, 1024, outdim_size]

    # the parameters for training the network
    learning_rate = 1e-3
    epoch_num = 100
    batch_size = 800

    # the regularization parameter of the network
    # seems necessary to avoid the gradient exploding especially when non-saturating activations are used
    reg_par = 1e-5

    # specifies if all the singular values should get used to calculate the correlation or just the top outdim_size ones
    # if one option does not work for a network or dataset, try the other one
    use_all_singular_values = False

    # if a linear CCA should get applied on the learned features extracted from the networks
    # it does not affect the performance on noisy MNIST significantly
    apply_linear_cca = True

    # end of parameters section
    ############

    # Each view is stored in a gzip file separately. They will get downloaded the first time the code gets executed.
    # Datasets get stored under the datasets folder of user's Keras folder
    # normally under [Home Folder]/.keras/datasets/
    data1 = load_data('noisymnist_view1.gz', 'https://www2.cs.uic.edu/~vnoroozi/noisy-mnist/noisymnist_view1.gz')
    data2 = load_data('noisymnist_view2.gz', 'https://www2.cs.uic.edu/~vnoroozi/noisy-mnist/noisymnist_view2.gz')

    # Building, training, and producing the new features by DCCA
    model = create_model(layer_sizes1, layer_sizes2, input_shape1, input_shape2,
                            learning_rate, reg_par, outdim_size, use_all_singular_values)
    model.summary()
    model = train_model(model, data1, data2, epoch_num, batch_size)
    new_data = test_model(model, data1, data2, outdim_size, apply_linear_cca)

    # Training and testing of SVM with linear kernel on the view 1 with new features
    [test_acc, valid_acc] = svm_classify(new_data, C=0.01)
    print("Accuracy on view 1 (validation data) is:", valid_acc * 100.0)
    print("Accuracy on view 1 (test data) is:", test_acc*100.0)

    # Saving new features in a gzip pickled file specified by save_to
    print('saving new features ...')
    f1 = gzip.open(save_to, 'wb')
    thepickle.dump(new_data, f1)
    f1.close()

## Train

In [ ]:
import gzip
import numpy as np

from keras.callbacks import ModelCheckpoint
from utils import load_data, svm_classify
from linear_cca import linear_cca
from models import create_model


def train_model(model, data1, data2, epoch_num, batch_size):
    """
    trains the model
    # Arguments
        data1 and data2: the train, validation, and test data for view 1 and view 2 respectively. data should be packed
        like ((X for train, Y for train), (X for validation, Y for validation), (X for test, Y for test))
        epoch_num: number of epochs to train the model
        batch_size: the size of batches
    # Returns
        the trained model
    """

    # Unpacking the data
    train_set_x1, train_set_y1 = data1[0]
    valid_set_x1, valid_set_y1 = data1[1]
    test_set_x1, test_set_y1 = data1[2]

    train_set_x2, train_set_y2 = data2[0]
    valid_set_x2, valid_set_y2 = data2[1]
    test_set_x2, test_set_y2 = data2[2]

    # best weights are saved in "temp_weights.hdf5" during training
    # it is done to return the best model based on the validation loss
    checkpointer = ModelCheckpoint(filepath="temp_weights.h5", verbose=1, save_best_only=True, save_weights_only=True)

    # used dummy Y because labels are not used in the loss function
    model.fit([train_set_x1, train_set_x2], np.zeros(len(train_set_x1)),
              batch_size=batch_size, epochs=epoch_num, shuffle=True,
              validation_data=([valid_set_x1, valid_set_x2], np.zeros(len(valid_set_x1))),
              callbacks=[checkpointer])

    model.load_weights("temp_weights.h5")

    results = model.evaluate([test_set_x1, test_set_x2], np.zeros(len(test_set_x1)), batch_size=batch_size, verbose=1)

    print('loss on test data: ', results)

    results = model.evaluate([valid_set_x1, valid_set_x2], np.zeros(len(valid_set_x1)), batch_size=batch_size, verbose=1)
    print('loss on validation data: ', results)
    return model

In [ ]:
def test_model(model, data1, data2, outdim_size, apply_linear_cca):
    """produce the new features by using the trained model
    # Arguments
        model: the trained model
        data1 and data2: the train, validation, and test data for view 1 and view 2 respectively.
            Data should be packed like
            ((X for train, Y for train), (X for validation, Y for validation), (X for test, Y for test))
        outdim_size: dimension of new features
        apply_linear_cca: if to apply linear CCA on the new features
    # Returns
        new features packed like
            ((new X for train - view 1, new X for train - view 2, Y for train),
            (new X for validation - view 1, new X for validation - view 2, Y for validation),
            (new X for test - view 1, new X for test - view 2, Y for test))
    """

    # producing the new features
    new_data = []
    for k in range(3):
        pred_out = model.predict([data1[k][0], data2[k][0]])
        r = int(pred_out.shape[1] / 2)
        new_data.append([pred_out[:, :r], pred_out[:, r:], data1[k][1]])

    # based on the DCCA paper, a linear CCA should be applied on the output of the networks because
    # the loss function actually estimates the correlation when a linear CCA is applied to the output of the networks
    # however it does not improve the performance significantly
    if apply_linear_cca:
        w = [None, None]
        m = [None, None]
        print("Linear CCA started!")
        w[0], w[1], m[0], m[1] = linear_cca(new_data[0][0], new_data[0][1], outdim_size)
        print("Linear CCA ended!")

        # Something done in the original MATLAB implementation of DCCA, do not know exactly why;)
        # it did not affect the performance significantly on the noisy MNIST dataset
        #s = np.sign(w[0][0,:])
        #s = s.reshape([1, -1]).repeat(w[0].shape[0], axis=0)
        #w[0] = w[0] * s
        #w[1] = w[1] * s
        ###

        for k in range(3):
            data_num = len(new_data[k][0])
            for v in range(2):
                new_data[k][v] -= m[v].reshape([1, -1]).repeat(data_num, axis=0)
                new_data[k][v] = np.dot(new_data[k][v], w[v])

    return new_data

## svm_classify

new features packed like：
1. ((new X for train - view 1, new X for train - view 2, Y for train),
2. (new X for validation - view 1, new X for validation - view 2, Y for validation),
3. (new X for test - view 1, new X for test - view 2, Y for test))

svm（train view1, Y for train) -> view 1 feature and Y for train image

svm predict(test view1) -> Y' and compare with Y for test image

In [ ]:
import gzip
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
import theano
from keras.utils.data_utils import get_file

def svm_classify(data, C):
    """
    trains a linear SVM on the data
    input C specifies the penalty factor of SVM
    """
    train_data, _, train_label = data[0]
    valid_data, _, valid_label = data[1]
    test_data, _, test_label = data[2]

    print('training SVM...')
    clf = svm.LinearSVC(C=C, dual=False)
    clf.fit(train_data, train_label.ravel())

    p = clf.predict(test_data)
    test_acc = accuracy_score(test_label, p)
    p = clf.predict(valid_data)
    valid_acc = accuracy_score(valid_label, p)

    return [test_acc, valid_acc]

# Deep canonically correlated autoencoders (DCCAE)

### multiple views of data at training time while only one view is available at test time:

1. audio + video
2. audio + atriculation
3. images + text
4. parallel text in two languages
5. words + context
6. document txt + text of inbound hyperlinks

This presents an opportunity to learn better representations by analyzing multip;e views simultaneously.

Typical approach: learning a feature transformation of the primary view that captures useful information from the second view using a paired two-view training set.

1. auotoencoders: the objective is to learn a compact representation that best reconstructs the inputs
2. CCA: learns features in two views that are maximally correlated

### Split autoencoders (SplitAE)

extract shared reprensentations by reconstructing both views from the one view that is available at test time. feature extraction network f is shared while the reconstruction network p and q are separate for each view.

object is the sum of reconstruction errors for the two views:
$min_{w_f,w_p,w_q} 1/N \sum_{i=1}^N (||x_i - p(f(x_i))||^2 + ||y_i-q(g(x_i))||^2)$
<img src="splitae.png" alt="Drawing" style="width: 300px;"/>

### DCCAE

object:
$min_{w_f,w_g,w_p,w_q,U,V} -1/N tr(U^Tf(X)g(Y)^TV) + \lambda/N \sum_{i=1}^N (||x_i - p(f(x_i))||^2 + ||y_i-q(g(x_i))||^2)$

<img src="dccae.png" alt="Drawing" style="width: 300px;"/>

CCA maximizes the mutual information between the projected views for certain distributions;

Autoencoder minimizes reconstruction error amounts to maximizing a lower bound on the mutual information between inputs and learned features.

#### Minimum-distance autoencoders (DistAE)
The CCA objective can be seen as minimizing the distance between the learned projections of the two views, while stisfying the whitening constraints for the projections. The constraints complicate the optimization of CCA-based objectives.

object:
$min_{w_f,w_g,w_p,w_q,U,V} -1/N \sum_{i=1}^N \frac{||f(x_i)-g(y_i)||^2}{||f(x_i)||^2+||g(y_i)||^2}+ \lambda/N \sum_{i=1}^N (||x_i - p(f(x_i))||^2 + ||y_i-q(g(x_i))||^2)$

# Deep Variational Canonical Correlation Analysis (VCCA)

A deep multi-view learning model that extends the latent variable model interpretation of linear CCA to nonlinear observation models parameterized by DNN.

Probabilistic latent variable model interpretation of CCA:
<img src="lat.png" alt="Drawing" style="width: 300px;"/>

x and y are linear functions of some random variable z. the prior distribution p(z) and conditional distributions p(x|z) and p(y|z) are Gaussian.

Jordan showed that E[z|x] lives in the same space as the linear CCA projection for x.

** This generative interpretation of CCA is often lost in its nonlinear extensions**.

### Disadvantage of DCCA:
1. Its objective couples all training samples together and is hard to optimize with SGD;
2. It focuses on extracting the shared information only, while there may also be useful view-specific information that we wish to retain;
3. It does not provide a model for generating samples from the latent space. Although Wang's DCCAE optimizes the combination of an autoencoder and cca objective, they found in practice, the cca objective dominate the reconstruction term, so the inputs are not reconstructed well.


## VCCA:
<img src="vcca.png" alt="Drawing" style="width: 300px;"/>

$p(x,y,z) = p(z)p(x|z)p(y|z), p(x,y) = \int p(x,y,z)dz$

The two views x and y are conditionally independent on the latent varianles z. Here, it consider non linear observation models $p_\theta(x|z;\theta_x)$ and $p_\theta(y|z;\theta_y)$. So marginal likelihood $p_\theta(x,y)$ does not have a closed form, and the inference problem $p_\theta(z|x)$ is also intractable.

Approximating $p_\theta(z|x) $ with the conditional density $q_\Phi(z|x;\Phi_z)$, where $\Phi_z$ is the collection of parameters of another DNN.

We can derive a lower bound on the marginal data log-likelihood and VCCA maximizes this variational lower bound on the data log-likelihood on the training set:

**object:**

$max_{\theta,\Phi} 1/N \sum_{i=1}^N L(x_i,y_i;\theta,\Phi)$

$logp_\theta(x,y) >= L(x,y;\theta,\Phi) := - D_{KL}(q_\Phi(z|x)||p(z)) + E{q_\Phi(z|x)}[logp_\theta(x|z)+logp_\theta(y|z)]$

**KL divergence term:**

$D_{KL}(q_\Phi(z|x)||p(z)) = -0.5 \sum_{j=1}^{d_z} (1+log \sigma_{ij}^2 -\sigma_{ij}^2 - \mu_{ij}^2)$

**expected log-likelihood term:**

$E{q_\Phi(z|x)}[logp_\theta(x|z)+logp_\theta(y|z)]\approx 1/L \sum_{l=1}^L log p_\theta(x_i|z_i^l) + log p_\theta(y_i|z_i^l)$

### Disadvantage of VCCA:
It assumes the common latent variables z are sufficient to generate the views. There might be large variations in the input space that can not be explained by the common variables.
## VCCA-private
<img src="vcca-p.png" alt="Drawing" style="width: 300px;"/>
$p(x,y,z,h_x,h_y) = p(z)p(h_x)p(h_y)p_\theta(x|z,h_x;\theta_x)p_\theta(y|z,h_y;\theta_y), p_\theta(x,y) = \int\int\int p_\theta(x,y,z,h_x,h_y)dzdh_xdh_y$

$q_\Phi(z,h_x,h_y|x,y) = q_\phi(z|x;\Phi_z)q_\Phi(h_x|x;\Phi_x)q_\Phi(h_y|y;\Phi_y).$

**object:**

$max_{\theta,\Phi} 1/N \sum_{i=1}^N L_{private}(x_i,y_i;\theta,\Phi)$

$logp_\theta(x,y) >= L(x,y;\theta,\Phi) := - D_{KL}(q_\Phi(z|x)||p(z)) -D_{KL}(q_\Phi(h_x|x)||p(h_x))-D_{KL}(q_\Phi(h_y|y)||p(h_y)) + E{q_\Phi(z|x)q_\Phi(h_x|x)}[logp_\theta(x|z,h_x)]+E_{q\Phi(z|x),q_\Phi(h_y|y)}[logp_\theta(y|z)]$


**The private variables contain little class information but mainly style information**

<img src="style.png" alt="Drawing" style="width: 300px;"/>

# VCCA-p for acoustic feature learning

Apply VCCA and VCCA-p on the X-ray Microbeam Database.

acoustic:
<img src="acous.png" alt="Drawing" style="width: 300px;"/>
articulatory:
<img src="art.png" alt="Drawing" style="width: 300px;"/>

## Generative adversrial training

We extend VCCA/VCCAP with generative adversarial training by viewing the basic VCCA/VCCAP model as a pair of generators, and adding two discriminators D1 and D2, one for each view, which try to distinguish the generated samples from training set unput samples.
<img src="vccapg.png" alt="Drawing" style="width: 300px;"/>

D1 optimizes:

$max_{D1} logD_1(x_i)+log(1-D1(x_i'))$, 

where x is the original input of the first view and x' is the corresponding reconstruction.

The generator optimize:

$max_{\theta,\Phi} L_{private}(x_i,y_i;\theta,\Phi) + \lambda_1log(D1(x_i'))+\lambda_2log(D2(y_i'))$

## Code for VCCA class in tensorflow

See https://github.com/edchengg/VCCA-StudyNotes/tree/master/qingming_tang-interspeech2017_vccap-fad4faebbc4b
